In [34]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [5]:
# Descrição das features:
# time           datetime  Data e Hora em que o veículo passa pelo pedágio;
# tollgate_id    string    Identificador do pedágio;
# direction      string    0: entra na rodovia pelo pedágio; 1: sai da rodovia pelo pedágio;
# vehicle_model  int       Um número que indica a capacidade do veículo;
# has_etc        string    Indica se o veículo possui ou não o sistema ETC; 0 - NÃO, 1 - SIM
# vehicle_type   string    0: veículo de passageiro; 1: veículo de carga
# weekday        int       Representa os dias da semana
# weekend        int       1: Para quando for fim de semana; 0: Para quando não for fim de semana

In [14]:
volume_df = pd.read_csv("dataset/volume(table 6)_test1.csv")
volume_df.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-10-18 07:59:04,2,0,1,1,NaN
1,2016-10-18 07:59:31,2,0,1,1,NaN
2,2016-10-18 07:59:50,2,0,1,1,NaN
3,2016-10-18 07:32:33,3,0,1,1,NaN
4,2016-10-18 07:32:46,3,0,1,1,NaN


In [15]:
#Caracteristicas dos dados
volume_df.info()
volume_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29441 entries, 0 to 29440
Data columns (total 6 columns):
time             29441 non-null object
tollgate_id      29441 non-null int64
direction        29441 non-null int64
vehicle_model    29441 non-null int64
has_etc          29441 non-null int64
vehicle_type     12715 non-null float64
dtypes: float64(1), int64(4), object(1)
memory usage: 1.3+ MB


,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
count,29441.000000,29441.000000,29441.000000,29441.000000,12715.000000
mean,2.151184,0.431881,1.218471,0.267450,0.198820
std,0.892908,0.495346,0.657182,0.442637,0.399128
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,0.000000
50%,2.000000,0.000000,1.000000,0.000000,0.000000
75%,3.000000,1.000000,1.000000,1.000000,0.000000
max,3.000000,1.000000,7.000000,1.000000,1.000000


In [12]:
#Retirando os valores nulos da coluna vehicle_type pelo modelo do veículo.
    #No vehicle_type indica 0 para veículo de passageiros e 1 para carga.
    #Poderíamos verificar a partir do modelo do veiculo, para veiculo com capacidade de até 4
    #Ficou definido que sera para passageiro, sendo maior que 4 será veiculo de carga

In [16]:
volume_df['vehicle_type'] = volume_df['vehicle_model'].apply(lambda x: 0 if x < 5 else 1)

In [17]:
volume_df.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-10-18 07:59:04,2,0,1,1,0
1,2016-10-18 07:59:31,2,0,1,1,0
2,2016-10-18 07:59:50,2,0,1,1,0
3,2016-10-18 07:32:33,3,0,1,1,0
4,2016-10-18 07:32:46,3,0,1,1,0


In [7]:
volume_df.tail()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
543694,2016-10-17 18:18:22,1,1,1,1,0
543695,2016-10-17 18:18:49,1,1,1,1,0
543696,2016-10-17 18:20:03,1,1,1,1,0
543697,2016-10-17 17:20:20,1,1,1,1,0
543698,2016-10-17 17:21:52,1,1,1,1,0


In [18]:
#Função que será usada para obter a janela de tempo de 20 minutos
def get_timewindow(t):
        time_window = 20
        if t.minute < time_window:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif t.minute < time_window*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window

def get_hour(t):
        return t.hour

In [19]:
#Ajustando o formato da coluna time
volume_df['time'] = pd.to_datetime(volume_df['time'], format = '%Y-%m-%d %H:%M:%S')

#Adiciona valores para os dias da semana
volume_df['weekday'] = volume_df['time'].dt.dayofweek + 1

#Classificar cada atributo de time aplicando a janela de tempo de vinte minutos
volume_df['t'] = volume_df['time'].dt.time

#Adicionando valores para saber se é referente a um fim de semana ou não
volume_df['weekend'] = volume_df['weekday'].apply(lambda x: 0 if x < 6 else 1)

volume_df['hour'] = volume_df['t'].apply(get_hour)

volume_df['date'] = volume_df['time'].dt.date


volume_df['time_window'] = volume_df['t'].apply(get_timewindow)
del volume_df['t']

volume_df.head()
volume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29441 entries, 0 to 29440
Data columns (total 11 columns):
time             29441 non-null datetime64[ns]
tollgate_id      29441 non-null int64
direction        29441 non-null int64
vehicle_model    29441 non-null int64
has_etc          29441 non-null int64
vehicle_type     29441 non-null int64
weekday          29441 non-null int64
weekend          29441 non-null int64
hour             29441 non-null int64
date             29441 non-null object
time_window      29441 non-null object
dtypes: datetime64[ns](1), int64(8), object(2)
memory usage: 2.5+ MB


In [20]:
#Salvando dados de treino
volume_df.to_csv('processed_test_volume2.csv', index = False)


In [46]:
#Fazendo o mesmo processo para os dados de teste

In [21]:
pd_volume_train = pd.read_csv('processed_training_volume2.csv')
pd_volume_test = pd.read_csv('processed_test_volume2.csv')
pd_volume_train = pd_volume_train.set_index(['time'])
pd_volume_test = pd_volume_test.set_index(['time'])
volume_train = pd_volume_train.groupby(['time_window','tollgate_id','direction','weekday', 'hour']).size().reset_index().rename(columns = {0:'volume'})
volume_test = pd_volume_test.groupby(['time_window','tollgate_id','direction','weekday', 'hour']).size().reset_index().rename(columns = {0:'volume'})

x = pd.Series(volume_train['time_window'].unique())
s = pd.Series(range(len(x)),index = x.values)
volume_train['window_n'] = volume_train['time_window'].map(s)
volume_test['window_n'] = volume_test['time_window'].map(s)
#volume_test.tail()
#volume_train.tail()
#volume_train.head()
volume_test.head()

,time_window,tollgate_id,direction,weekday,hour,volume,window_n
0,"[06:00:00,06:20:00)",1,0,1,6,18,18
1,"[06:00:00,06:20:00)",1,0,2,6,13,18
2,"[06:00:00,06:20:00)",1,0,3,6,12,18
3,"[06:00:00,06:20:00)",1,0,4,6,8,18
4,"[06:00:00,06:20:00)",1,0,5,6,8,18


In [36]:
def feature_format():
    pd_volume_train = pd.read_csv('processed_training_volume2.csv')
    pd_volume_test = pd.read_csv('processed_test_volume2.csv')
    pd_volume_train = pd_volume_train.set_index(['time'])
    pd_volume_test = pd_volume_test.set_index(['time'])
    volume_train = pd_volume_train.groupby(['time_window','tollgate_id','direction','weekday', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    volume_test = pd_volume_test.groupby(['time_window','tollgate_id','direction','weekday', 'hour']).size().reset_index().rename(columns = {0:'volume'})
                    
    x = pd.Series(volume_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    volume_train['window_n'] = volume_train['time_window'].map(s)
    volume_test['window_n'] = volume_test['time_window'].map(s)
    #volume_test.tail()
    
    feature_train = volume_train.drop('volume', axis = 1).set_index(['time_window'])
    feature_test = volume_test.drop('volume',axis = 1).set_index(['time_window'])
    values_train = volume_train['volume'].values
    values_test = volume_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [37]:
x_train, x_test, y_train, y_test = feature_format()

In [24]:
x_train

,tollgate_id,direction,weekday,hour,window_n
time_window,,,,,
"[00:00:00,00:20:00)",1,0,1,0,0
"[00:00:00,00:20:00)",1,0,2,0,0
"[00:00:00,00:20:00)",1,0,3,0,0
"[00:00:00,00:20:00)",1,0,4,0,0
"[00:00:00,00:20:00)",1,0,5,0,0
"[00:00:00,00:20:00)",1,0,6,0,0
"[00:00:00,00:20:00)",1,0,7,0,0
"[00:00:00,00:20:00)",1,1,1,0,0
"[00:00:00,00:20:00)",1,1,2,0,0


In [25]:
y_train

array([70, 48, 58, ..., 43, 31, 42])

In [26]:
x_test.head()

,tollgate_id,direction,weekday,hour,window_n
time_window,,,,,
"[06:00:00,06:20:00)",1,0,1,6,18
"[06:00:00,06:20:00)",1,0,2,6,18
"[06:00:00,06:20:00)",1,0,3,6,18
"[06:00:00,06:20:00)",1,0,4,6,18
"[06:00:00,06:20:00)",1,0,5,6,18


In [27]:
y_test

array([ 18,  13,  12,   8,   8,   7,  13,  31,  37,  48,  46,  33,  31,
        26,  27,  24,  26,  21,  16,  22,   8,  41,  30,  24,  37,  22,
        27,  18,  23,  23,  21,  16,  21,  19,   9,  21,  17,  16,  16,
        13,   8,  10,  57,  47,  28,  27,  39,  36,  50,  39,  36,  45,
        23,  24,  22,  11,  53,  42,  57,  44,  49,  37,  29,  41,  29,
        26,  49,  28,  18,  19,  24,  21,  17,  23,  21,  22,   8,  59,
        72,  57,  55,  54,  56,  33,  52,  46,  47,  42,  39,  43,  25,
       105,  85,  75,  73,  66,  51,  43,  60,  50,  43,  37,  42,  25,
        33,  35,  31,  22,  22,  32,  14,  17,  76,  68,  67,  71,  65,
        49,  45, 104,  83,  87,  76,  66,  24,  29, 150, 114, 119, 114,
       116,  76,  51,  75,  56,  53,  47,  54,  38,  36,  34,  28,  38,
        33,  41,  27,  24, 116,  94,  98,  88, 102,  44,  41, 111,  87,
        95,  89,  77,  49,  35, 148, 131, 137, 123, 135,  89,  71, 101,
        77,  63,  60,  76,  39,  53,  43,  47,  41,  37,  38,  3

In [77]:
#Função que calcula a média quadrática
def rmse(predictions, targets):
    differences = predictions - targets
    differences_squared = differences ** 2
    mean_of_differences_squared = differences_squared.mean()
    rmse_val = np.sqrt(mean_of_differences_squared)
    return rmse_val

In [78]:
#Função que calcula o MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [29]:
#Executando algoritmo de regressão linear
model1 = LinearRegression()
model1.fit(x_train, y_train)
pred_y1 = model1.predict(x_test)
pred_y1

array([199.43103438, 194.19812541, 188.96521644, 183.73230747,
       178.4993985 , 173.26648953, 168.03358056, 186.40155535,
       181.16864638, 175.93573741, 170.70282844, 165.46991947,
       160.23701051, 155.00410154, 222.66168914, 217.42878017,
       212.1958712 , 206.96296223, 201.73005326, 196.49714429,
       191.26423532, 245.8923439 , 240.65943493, 235.42652596,
       230.19361699, 224.96070802, 219.72779906, 214.49489009,
       232.86286487, 227.62995591, 222.39704694, 217.16413797,
       211.931229  , 206.69832003, 201.46541106, 199.0671165 ,
       193.83420753, 188.60129856, 183.36838959, 178.13548062,
       172.90257165, 167.66966268, 186.03763747, 180.8047285 ,
       175.57181953, 170.33891056, 165.10600159, 159.87309262,
       154.64018365, 222.29777126, 217.06486229, 211.83195332,
       206.59904435, 201.36613538, 196.13322641, 190.90031744,
       245.52842602, 240.29551705, 235.06260808, 229.82969911,
       224.59679014, 219.36388117, 214.1309722 , 232.49

In [81]:
#Algoritmo Regressão Linear com Gradiente Descendente
class LinearRegressionGD(object):

    def __init__(self, eta=0.001, n_iter=20):
        self.eta = eta
        self.n_iter = n_iter

    def fit(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        return self.net_input(X)

In [89]:
model2 = LinearRegressionGD(eta = 0.0009, n_iter = 40)
model2.fit(x_train, y_train)
pred_y2 = model2.predict(x_test)

In [32]:
#Algoritmo Regressão Linear com Gradiente Descendente Estocrástico
model3 = SGDRegressor(eta0=0.0009, max_iter=40)
model3.fit(x_train, y_train)
pred_y3 = model3.predict(x_test)

In [103]:
mean_absolute_percentage_error(y_test, pred_y1)

344.6219144002485

In [102]:
print(  rmse(pred_y1, y_test),"\n", #Algoritmo 1
        rmse(pred_y2, y_test),"\n", #Algoritmo 2
        rmse(pred_y3, y_test) #Algoritmo 3
     )

150.6964566871464 
 5.701458141552023e+145 
 187.39084578228082


In [33]:
mean_squared_error(pred_y3, y_test)

28344.218948382113

In [40]:
rng = np.random.RandomState(1)
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 21),
                         n_estimators=11, random_state = rng)

regr.fit(x_train, y_train)

y_pred = regr.predict(x_test)

mape = np.mean(np.abs((y_pred - y_test)/y_test))

print (mape)

4.002390009178407
